In [7]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import json
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
import requests
import random
import time

In [8]:
df = pd.read_pickle('data/yelpexport_new.pkl')
df = df.fillna(0)
print df.shape
df.head()

(341, 18)


,arts,shopping,beautysvc,financialservices,restaurants,zipcode,religiousorgs,food,health,massmedia,pets,professional,active,hotelstravel,education,publicservicesgovt,realestate,nightlife
0,0.015132,0.142411,0.107582,0.020616,0.257038,11368,0.008332,0.141767,0.087977,0.001501,0.022239,0.030847,0.030847,0.039822,0.022270,0.010047,0.026926,0.034645
0,0.010592,0.156386,0.095950,0.014953,0.327103,79936,0.008100,0.116511,0.082243,0.000000,0.019938,0.017445,0.036137,0.021184,0.012461,0.007477,0.023676,0.049844
0,0.018508,0.166826,0.098629,0.035725,0.131185,90650,0.010213,0.083205,0.126516,0.000959,0.026846,0.066781,0.041394,0.045604,0.039518,0.007754,0.086957,0.013381
0,0.035742,0.171011,0.095117,0.017556,0.213251,11226,0.006177,0.120098,0.112266,0.003892,0.015950,0.035878,0.038287,0.032579,0.021509,0.007993,0.027761,0.044933
0,0.037747,0.171274,0.093309,0.017269,0.210591,11220,0.005652,0.117650,0.112012,0.004055,0.016054,0.038006,0.038662,0.033911,0.021802,0.008095,0.028204,0.045706


In [7]:
# with help from http://planspace.org/20150423-forward_selection_with_statsmodels/
import statsmodels.formula.api as smf

def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response, ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < (best_new_score * 1):
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response, ' + '.join(selected))
    
    model = smf.ols(formula, data).fit()
    return model

In [11]:
response=pd.read_pickle('data/housing_rental_ratio.pkl')
response=response.rename(columns={'RegionName':'zipcode'})
response['zipcode']=response['zipcode'].astype(str)
response['zipcode'] = [(5-len(zipcode))*'0' + zipcode if len(zipcode) < 5 else zipcode for zipcode in response['zipcode']]
response.head()

,zipcode,r_agg_2011,r_agg_2015_adj,r_pct_delta,h_agg_2011,h_agg_2015_adj,h_pct_delta,ratio_2011,ratio_2015,ratio_pct_delta
0,11368,1.868000,2.188278,1.171455,373.833333,431.274105,1.153653,16.677076,16.423646,0.984804
1,79936,0.744167,0.694379,0.933096,80.333333,75.060164,0.934359,8.995894,9.008072,1.001354
2,60629,1.197000,1.180207,0.985971,107.166667,113.430961,1.058454,7.460782,8.009253,1.073514
3,90650,1.524333,1.640026,1.075898,228.750000,292.549418,1.278905,12.505467,14.865076,1.188686
4,11226,1.014000,1.277783,1.260141,309.166667,416.876846,1.348389,25.408174,27.187509,1.070030


In [12]:
zipCodeStrings = df.zipcode
all_features = df.astype('float')

In [13]:
from sklearn.cross_validation import train_test_split
# split into training and test sets
itrain, itest = train_test_split(xrange(df.shape[0]), train_size=0.8)

mask=np.ones(df.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)

print 'samples in training set = %s \nsamples in test set = %s'%(mask.sum(), mask.shape[0] - mask.sum())

samples in training set = 272 
samples in test set = 69


In [15]:
train_df = df[mask]
test_df = df[~mask]
train_df.shape, test_df.shape

((272, 18), (69, 18))

In [18]:
train_ratio_2015 = pd.merge(train_df, response[['zipcode','ratio_2011','ratio_2015']], on='zipcode')
test_ratio_2015 = pd.merge(test_df, response[['zipcode','ratio_2011','ratio_2015']], on='zipcode')

In [21]:
train_ratio_2015_baseline = forward_selected(train_ratio_2015[['ratio_2011', 'ratio_2015']], 'ratio_2015')
train_ratio_2015_baseline.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             ratio_2015   R-squared:                       0.839
Model:                            OLS   Adj. R-squared:                  0.838
Method:                 Least Squares   F-statistic:                     1404.
Date:                Thu, 10 Dec 2015   Prob (F-statistic):          5.67e-109
Time:                        20:16:00   Log-Likelihood:                -551.19
No. Observations:                 272   AIC:                             1106.
Df Residuals:                     270   BIC:                             1114.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      2.2663      0.308      7.347      0.000         1.659     2.874
ratio_2011     0.9238      0.025     37.469      0.000         0.875     0.972
==============================================================================
Omnibus:                       36.036   Durbin-Watson:                   2.119
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                9.915
Skew:                          -0.041   Prob(JB):                      0.00703
Kurtosis:                       2.068   Cond. No.                         34.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
train_ratio_2015_model = forward_selected(train_ratio_2015.drop('zipcode',1), 'ratio_2015')
ratio_2015_model_features = train_ratio_2015_model.params.index.values.tolist()
train_ratio_2015_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             ratio_2015   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.929
Method:                 Least Squares   F-statistic:                     298.5
Date:                Thu, 10 Dec 2015   Prob (F-statistic):          5.31e-144
Time:                        20:15:55   Log-Likelihood:                -432.60
No. Observations:                 272   AIC:                             891.2
Df Residuals:                     259   BIC:                             938.1
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
Intercept            -0.3850      0.912     -0.422      0.673        -2.182     1.412
ratio_2011            0.9112      0.017     52.283      0.000         0.877     0.946
realestate           23.5505      2.989      7.880      0.000        17.666    29.435
financialservices    48.7506     13.198      3.694      0.000        22.762    74.739
pets                -47.3446     10.435     -4.537      0.000       -67.893   -26.796
nightlife           -49.5769      8.483     -5.844      0.000       -66.281   -32.873
arts                 24.7866     10.414      2.380      0.018         4.279    45.294
beautysvc            13.7284      5.959      2.304      0.022         1.994    25.463
shopping              9.8772      3.639      2.714      0.007         2.711    17.044
education           -10.9835      4.345     -2.528      0.012       -19.540    -2.427
massmedia           159.4860     76.197      2.093      0.037         9.442   309.530
active              -10.6179      6.874     -1.545      0.124       -24.154     2.919
religiousorgs       -39.7357     29.228     -1.359      0.175       -97.291    17.820
==============================================================================
Omnibus:                        8.942   Durbin-Watson:                   2.027
Prob(Omnibus):                  0.011   Jarque-Bera (JB):               12.341
Skew:                           0.238   Prob(JB):                      0.00209
Kurtosis:                       3.929   Cond. No.                     1.30e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.3e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
ratio_2015_model_features = [el.replace('[T.True]','') for el in ratio_2015_model_features if el != 'Intercept'] + ['ratio_2015']
ratio_2015_model_features

['ratio_2011',
 'realestate',
 'financialservices',
 'pets',
 'nightlife',
 'arts',
 'beautysvc',
 'shopping',
 'education',
 'massmedia',
 'active',
 'religiousorgs',
 'ratio_2015']

In [23]:
# feature selected training, testing
fs_train_trans_df_ratio_2015 = train_ratio_2015[ratio_2015_model_features].dropna()
fs_test_trans_df_ratio_2015 = test_ratio_2015[ratio_2015_model_features].dropna()
print fs_train_trans_df_ratio_2015.shape, fs_test_trans_df_ratio_2015.shape

(272, 13) (69, 13)


#lasso

In [24]:
from sklearn.linear_model import LassoCV
# ratio 2015 only
start_time = time.time()

X_train = fs_train_trans_df_ratio_2015.drop('ratio_2015',1).values * 1
Y_train = fs_train_trans_df_ratio_2015['ratio_2015'].values * 1
X_test = fs_test_trans_df_ratio_2015.drop('ratio_2015',1).values * 1
Y_test = fs_test_trans_df_ratio_2015['ratio_2015'].values * 1

alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
alphas_temp = [0.001, 0.01]
clf_lasso_model = LassoCV(alphas = alphas).fit(X_train, Y_train)
best_alpha_lasso = clf_lasso_model.alpha_

print '----- %s seconds to run ----- \n---- best alpha = %s ----'%(time.time() - start_time, best_alpha_lasso)

----- 0.0139520168304 seconds to run ----- 
---- best alpha = 0.001 ----


//anaconda/lib/python2.7/site-packages/sklearn/linear_model/base.py:424: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if precompute == 'auto':


In [26]:
# calculate accuracy
start_time = time.time()

training_accuracy = clf_lasso_model.score(X_train, Y_train)
test_accuracy = clf_lasso_model.score(X_test, Y_test)
print '#### based on the best Linear Regression with Lasso Regularization ####'
print "R-squared on training data: %0.2f" % (training_accuracy)
print "R-squared on test data:     %0.2f" % (test_accuracy)
print '----- %s seconds to run -----'%(time.time() - start_time)

#### based on the best Linear Regression with Lasso Regularization ####
R-squared on training data: 0.93
R-squared on test data:     0.94
----- 0.00116205215454 seconds to run -----


#ridge

In [27]:
from sklearn.linear_model import RidgeCV

#ratio only
start_time = time.time()

X_train = fs_train_trans_df_ratio_2015.drop('ratio_2015',1).values * 1
Y_train = fs_train_trans_df_ratio_2015['ratio_2015'].values * 1
X_test = fs_test_trans_df_ratio_2015.drop('ratio_2015',1).values * 1
Y_test = fs_test_trans_df_ratio_2015['ratio_2015'].values * 1

alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
clf_ridge_model = RidgeCV(alphas = alphas).fit(X_train, Y_train)
best_alpha_ridge = clf_ridge_model.alpha_

print '----- %s seconds to run ----- \n---- best alpha = %s ----'%(time.time() - start_time, best_alpha_ridge)

----- 0.00356316566467 seconds to run ----- 
---- best alpha = 0.001 ----


In [28]:
start_time = time.time()
training_accuracy = clf_ridge_model.score(X_train, Y_train)
test_accuracy = clf_ridge_model.score(X_test, Y_test)
print '#### based on the best Linear Regression with Ridge Regularization ####'
print "R-squared on training data: %0.2f" % (training_accuracy)
print "R-squared on test data:     %0.2f" % (test_accuracy)
print '----- %s seconds to run -----'%(time.time() - start_time)

#### based on the best Linear Regression with Ridge Regularization ####
R-squared on training data: 0.93
R-squared on test data:     0.94
----- 0.000977039337158 seconds to run -----


In [29]:
len(pd.read_pickle('data/yelpexport_new.pkl')['zipcode'])

341